In [22]:
import argparse
import random
import time

import numpy as np
import torch
import Custom.Trainer
import Custom.Trainer.CustomTrainer
from rPPG_Toolbox.config import _update_config_from_file,_C,update_config
from rPPG_Toolbox.dataset import data_loader
from rPPG_Toolbox.neural_methods import trainer
from rPPG_Toolbox.unsupervised_methods.unsupervised_predictor import unsupervised_predict
from torch.utils.data import DataLoader
import Custom

In [23]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
RANDOM_SEED = 100
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Create a general generator for use with the validation dataloader,
# the test dataloader, and the unsupervised dataloader
general_generator = torch.Generator()
general_generator.manual_seed(RANDOM_SEED)
# Create a training generator to isolate the train dataloader from
# other dataloaders and better control non-deterministic behavior
train_generator = torch.Generator()
train_generator.manual_seed(RANDOM_SEED)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [24]:
config_file = rf'.\configs\train_configs\SCAMPS_SCAMPS_UBFC-rPPG_DEEPPHYS_BASIC.yaml'

config = _C.clone()
args = dotdict({'config_file':config_file})
update_config(config, args)
print(config)

=> Merging a config file from .\configs\train_configs\SCAMPS_SCAMPS_UBFC-rPPG_DEEPPHYS_BASIC.yaml
BASE: ['']
DEVICE: cpu
INFERENCE:
  BATCH_SIZE: 4
  EVALUATION_METHOD: FFT
  EVALUATION_WINDOW:
    USE_SMALLER_WINDOW: False
    WINDOW_SIZE: 10
  MODEL_PATH: 
LOG:
  PATH: runs/exp
MODEL:
  BIGSMALL:
    FRAME_DEPTH: 3
  DROP_RATE: 0.2
  EFFICIENTPHYS:
    FRAME_DEPTH: 10
  MODEL_DIR: runs/exp\SCAMPS_SizeW72_SizeH72_ClipLength180_DataTypeDiffNormalized_Standardized_DataAugNone_LabelTypeDiffNormalized_Crop_faceTrue_BackendHC_Large_boxTrue_Large_size1.5_Dyamic_DetFalse_det_len30_Median_face_boxFalse\PreTrainedModels
  NAME: DeepPhys
  PHYSFORMER:
    DIM: 96
    FF_DIM: 144
    NUM_HEADS: 4
    NUM_LAYERS: 12
    PATCH_SIZE: 4
    THETA: 0.7
  PHYSNET:
    FRAME_NUM: 64
  RESUME: 
  TSCAN:
    FRAME_DEPTH: 10
  iBVPNet:
    CHANNELS: 3
    FRAME_NUM: 160
NUM_OF_GPU_TRAIN: 1
TEST:
  DATA:
    BEGIN: 0.0
    CACHED_PATH: .\data\Cached\UBFC-rPPG\UBFC-rPPG_SizeW72_SizeH72_ClipLength180_DataTyp

In [25]:
train_data_loader = data_loader.SCAMPSLoader.SCAMPSLoader


train_data_loader = train_data_loader(
    name="train",
    data_path=config.TRAIN.DATA.DATA_PATH,
    config_data=config.TRAIN.DATA)

data_loader_dict = dict()
data_loader_dict['train'] = DataLoader(
    dataset=train_data_loader,
    num_workers=16,
    batch_size=config.TRAIN.BATCH_SIZE,
    shuffle=True,
    worker_init_fn=seed_worker,
    generator=train_generator
)

Cached Data Path .\data\Cashed\SCAMPS\Train\SCAMPS_SizeW72_SizeH72_ClipLength180_DataTypeDiffNormalized_Standardized_DataAugNone_LabelTypeDiffNormalized_Crop_faceTrue_BackendHC_Large_boxTrue_Large_size1.5_Dyamic_DetFalse_det_len30_Median_face_boxFalse

File List Path .\data\Cashed\SCAMPS\Train\DataFileLists\SCAMPS_SizeW72_SizeH72_ClipLength180_DataTypeDiffNormalized_Standardized_DataAugNone_LabelTypeDiffNormalized_Crop_faceTrue_BackendHC_Large_boxTrue_Large_size1.5_Dyamic_DetFalse_det_len30_Median_face_boxFalse_0.0_1.0.csv
 train Preprocessed Dataset Length: 18



In [26]:
valid_data_loader = data_loader.SCAMPSLoader.SCAMPSLoader


valid_data_loader = valid_data_loader(
    name="valid",
    data_path=config.VALID.DATA.DATA_PATH,
    config_data=config.VALID.DATA)

data_loader_dict = dict()
data_loader_dict['valid'] = DataLoader(
    dataset=valid_data_loader,
    num_workers=16,
    batch_size=config.TRAIN.BATCH_SIZE,
    shuffle=True,
    worker_init_fn=seed_worker,
    generator=general_generator
)

Preprocessing dataset...


 50%|█████     | 1/2 [00:27<00:27, 27.67s/it]

Total Number of raw files preprocessed: 2

Cached Data Path .\data\Cashed\SCAMPS\Val

File List Path PreprocessedData\DataFileLists
 valid Preprocessed Dataset Length: 6



In [27]:
test_data_loader = data_loader.UBFCrPPGLoader.UBFCrPPGLoader


test_data_loader = test_data_loader(
    name="test",
    data_path=config.TEST.DATA.DATA_PATH,
    config_data=config.TEST.DATA)

data_loader_dict = dict()
data_loader_dict['test'] = DataLoader(
    dataset=test_data_loader,
    num_workers=16,
    batch_size=config.INFERENCE.BATCH_SIZE,
    shuffle=True,
    worker_init_fn=seed_worker,
    generator=general_generator
)

Preprocessing dataset...


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:32<00:32, 32.91s/it]

Total Number of raw files preprocessed: 2

Cached Data Path .\data\Cached\UBFC-rPPG\UBFC-rPPG_SizeW72_SizeH72_ClipLength180_DataTypeDiffNormalized_Standardized_DataAugNone_LabelTypeDiffNormalized_Crop_faceTrue_BackendHC_Large_boxTrue_Large_size1.5_Dyamic_DetFalse_det_len30_Median_face_boxFalse

File List Path .\data\Cached\UBFC-rPPG\DataFileLists\UBFC-rPPG_SizeW72_SizeH72_ClipLength180_DataTypeDiffNormalized_Standardized_DataAugNone_LabelTypeDiffNormalized_Crop_faceTrue_BackendHC_Large_boxTrue_Large_size1.5_Dyamic_DetFalse_det_len30_Median_face_boxFalse_0.0_1.0.csv
 test Preprocessed Dataset Length: 18



In [28]:
model_trainer = model_trainer = trainer.DeepPhysTrainer.DeepPhysTrainer(config, data_loader_dict)

AttributeError: module 'rPPG_Toolbox.neural_methods.trainer' has no attribute 'DeepPhysTrainer'

In [ ]:
model_trainer.train(data_loader_dict)

In [ ]:
model_trainer.test(data_loader_dict)